In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.16.144:36441
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
## path for mdules
import sys
sys.path.insert(0,"/home/jovyan/xscale")
import xscale


sys.path.insert(0,"/home/jovyan/eNATL60-plots-paper/common-lib/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [4]:
zmap_grid = fs.get_mapper('pangeo-meom/eNATL60-I/eNATL60-mesh-mask')
ds_grid = xr.open_zarr(zmap_grid)


distributed.scheduler - INFO - Register tcp://10.32.11.26:39797
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.11.26:39797
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.19.9:39869
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.19.9:39869
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.12.33:39905
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.12.33:39905
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.11.27:44665
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.11.27:44665
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.10.18:37067
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.10.18:37067
distributed.cor

In [5]:
zmap_u = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSU-1h')
ds_u = xr.open_zarr(zmap_u)


distributed.scheduler - INFO - Register tcp://10.32.10.21:45845
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.10.21:45845
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.19.11:37117
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.19.11:37117
distributed.core - INFO - Starting established connection


In [6]:
zmap_v = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSV-1h')
ds_v = xr.open_zarr(zmap_v)


In [7]:
navlat= ds_grid['nav_lat']
navlon= ds_grid['nav_lon']
e1u=ds_grid.e1u[0]
e1v=ds_grid.e1v[0]
e2u=ds_grid.e2u[0]
e2v=ds_grid.e2v[0]
ff=ds_grid.ff[0]


In [8]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=90, dim=['x', 'y'], n=[90, 90])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
def curl(u,v,e1v,e2u,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    #Computation of the vorticity divided by f
    fff=1/ff
    curl=(dxvt-dyut)*fff
    return curl

In [10]:
    print('Select dates in zarr')
    u=ds_u.sel(time_counter=slice('2010-03-01','2010-03-31'))['sozocrtx']
    v=ds_v.sel(time_counter=slice('2010-03-01','2010-03-31'))['somecrty']


Select dates in zarr


In [11]:
u

<xarray.DataArray 'sozocrtx' (time_counter: 744, y: 4729, x: 8354)>
dask.array<getitem, shape=(744, 4729, 8354), dtype=float32, chunksize=(24, 120, 120), chunktype=numpy.ndarray>
Coordinates:
    nav_lat        (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-03-01T00:30:00 ... 2010-03-31T23:30:00
Dimensions without coordinates: y, x
Attributes:
    cell_methods:        time: mean (interval: 40 s)
    interval_operation:  40 s
    interval_write:      1 h
    long_name:           ocean surface current along i-axis
    online_operation:    average
    units:               m/s

In [ ]:
%%time

chunkt=10
chunkx=400
chunky=np.int(np.floor(125*(1000**2)/(chunkt*chunkx*u[0].dtype.itemsize)))
u_rechunk=u.chunk({'time_counter':chunkt,'y':chunky,'x':chunkx})
v_rechunk=v.chunk({'time_counter':chunkt,'y':chunky,'x':chunkx})
curl_surf=curl(u_rechunk,v_rechunk,e1v,e2u,ff)
curl_SS=filt(curl_surf)
curl_LS=curl_surf-curl_SS
hpcurl=curl_SS
hpcurl2 = hpcurl ** 2
hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
hpcurl2m.load()

distributed.scheduler - INFO - Register tcp://10.32.22.2:37987
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.22.2:37987
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.22.3:43873
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.22.3:43873
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.30.3:34395
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.30.3:34395
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.30.4:37869
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.30.4:37869
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.48.3:42577
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.48.3:42577
distributed.core - INFO